In [26]:
from pathlib import Path
import os
import json

In [27]:
home = Path(os.getenv('HOME'))
wikiextract_path = home / 'wikiextract'
eng_words_path = home / 'eng_words'
references_path = home / 'references'
wikiextract_path.exists()


True

In [28]:
eng_words = eng_words_path.open("w")
references = references_path.open("w")


In [29]:
def extract_word_data(word_data):
  d = {
    "word": word_data['word'],
    "senses": word_data['senses'],
    "pos": word_data['pos'],
  }

  sound_data = {}
  if 'sounds' in word_data:
    for s in word_data['sounds']:
      if 'tags' in s and "US" not in s['tags']:
        continue
      if 'ipa' in s:
        sound_data['ipa'] = s['ipa']
      if 'mp3_url' in s:
        sound_data['mp3_url'] = s['mp3_url']
      if 'ogg_url' in s:
        sound_data['ogg_url'] = s['ogg_url']

  d['sounds'] = sound_data
  return d

saved = 0
redirects = 0
discarded = 0
try:
  with wikiextract_path.open("r") as f:
    for idx, line in enumerate(f):
      if (idx + 1) % 1_000_000 == 0:
        print(f"{saved=} {redirects=} {discarded=}, total={idx+1}")
      data = json.loads(line)
      if "redirect" in data:
        references.write(line + "\n")
        redirects += 1
      elif "lang_code" in data and data['lang_code'] == "en":
        eng_words.write(json.dumps(extract_word_data(data)) + "\n")
        saved +=1
      else:
        discarded += 1
finally:
  eng_words.close()
  references.close()
